# Baseline

### Data preprocessing

```
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003-inv.jpg
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003-aughor-inv.jpg
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003-aughor.jpg
27042021_óÑαΓ«½ÑΓ¡¿¿» 12_30@ds-0003.jpg
```

In [14]:
import os
import random
import shutil
from collections import defaultdict

In [41]:
def collect_files(base_dir):
   file_dict = defaultdict(list)
   for root, dirs, files in os.walk(base_dir):
       for file in files:
           if file.endswith(('.jpg', '.txt')):
               base_name = "".join(file.split('-')[:-1])  # Убираем аугментации
               file_dict[base_name].append(os.path.join(root, file))
   return file_dict

def split_dataset(file_dict, dataset_size, train_ratio=0.8, valid_ratio=0.1):
   # Подготовка путей для dataset
   train_dir = 'baseline_dataset/train'
   valid_dir = 'baseline_dataset/valid'
   test_dir = 'baseline_dataset/test'
   
   # Создаём директории
   for dir_path in [train_dir, valid_dir, test_dir]:
       os.makedirs(os.path.join(dir_path, 'images'), exist_ok=True)
       os.makedirs(os.path.join(dir_path, 'labels'), exist_ok=True)
   
   # Подсчитаем сколько нужно изображений для каждой части
   num_train = int(dataset_size * train_ratio)
   num_valid = int(dataset_size * valid_ratio)
   num_test = dataset_size - num_train - num_valid

   print(num_train, num_valid, num_test)
    
   # Сортировка по классам
   keys = list(file_dict.keys())
   random.shuffle(keys)  # Перемешиваем список

   train_set, valid_set, test_set = set(), set(), set()
    
   for _ in range(num_train):
       key = keys.pop()
       train_set.update(file_dict[key])
       
   for _ in range(num_valid):
       key = keys.pop()
       valid_set.update(file_dict[key])

   for _ in range(num_test):
       key = keys.pop()
       test_set.update(file_dict[key])

   return train_set, valid_set, test_set

def copy_files(file_set, dest_dir):
   for file in file_set:
       sub_dir = 'images' if file.endswith('.jpg') else 'labels'
       dest_path = os.path.join(dest_dir, sub_dir, os.path.basename(file))
       shutil.copy(file, dest_path)

In [44]:
# dataset_size = 3000

# # Сбор данных для каждого класса
# plane_files = collect_files('dataset/photos/0-plane')
# helicopter_files = collect_files('dataset/photos/1-helicopter')

# # Объединяем оба класса
# all_files = {**plane_files, **helicopter_files}

# # Разбиваем на train/valid/test
# train_set, valid_set, test_set = split_dataset(all_files, dataset_size)
# print(len(train_set), len(valid_set), len(test_set))
# # Копируем файлы в соответствующие директории
# copy_files(train_set, 'baseline_dataset/train')
# copy_files(valid_set, 'baseline_dataset/valid')
# copy_files(test_set, 'baseline_dataset/test')

2400 300 300
7182 894 914


In [45]:
def collect_backgrounds(background_dir):
    backgrounds = []
    for root, dirs, files in os.walk(background_dir):
        for file in files:
            if file.endswith('.jpg'):
                backgrounds.append(os.path.join(root, file))
    return backgrounds

def split_backgrounds(backgrounds, train_ratio=0.8, valid_ratio=0.1):
    random.shuffle(backgrounds)
    num_background_train = int(len(backgrounds) * train_ratio)
    num_background_valid = int(len(backgrounds) * valid_ratio)
    
    background_train = backgrounds[:num_background_train]
    background_valid = backgrounds[num_background_train:num_background_train + num_background_valid]
    background_test = backgrounds[num_background_train + num_background_valid:]
    
    return background_train, background_valid, background_test

def copy_backgrounds(background_set, dest_dir):
    for background in background_set:
        dest_path = os.path.join(dest_dir, 'images', os.path.basename(background))
        shutil.copy(background, dest_path)

In [46]:
# backgrounds = collect_backgrounds('dataset/photos/backgrounds/images')

# # Разбиваем background снимки на train/valid/test
# background_train, background_valid, background_test = split_backgrounds(backgrounds)

# # Копируем background изображения в существующие директории
# copy_backgrounds(background_train, 'baseline_dataset/train')
# copy_backgrounds(background_valid, 'baseline_dataset/valid')
# copy_backgrounds(background_test, 'baseline_dataset/test')

In [47]:
# len(backgrounds)

216

In [48]:
def count_files(directory):
    return len([file for file in os.listdir(directory) if os.path.isfile(os.path.join(directory, file))])

# Папки, которые нужно проверить
directories = {
    'train': ('baseline_dataset/train/images', 'baseline_dataset/train/labels'),
    'valid': ('baseline_dataset/valid/images', 'baseline_dataset/valid/labels'),
    'test': ('baseline_dataset/test/images', 'baseline_dataset/test/labels')
}

for key, (img_dir, label_dir) in directories.items():
    img_count = count_files(img_dir)
    label_count = count_files(label_dir)
    print(f"{key.capitalize()}:")
    print(f"  Количество файлов в images: {img_count}")
    print(f"  Количество файлов в labels: {label_count}")

Train:
  Количество файлов в images: 3763
  Количество файлов в labels: 3591
Valid:
  Количество файлов в images: 468
  Количество файлов в labels: 447
Test:
  Количество файлов в images: 480
  Количество файлов в labels: 457


## Training

In [52]:
import torch
torch.cuda.is_available()

True

In [3]:
from ultralytics import YOLO

In [56]:
model = YOLO("yolo11s.pt")
model.train(data="data.yaml", epochs=100, batch=-1, plots=True, save=True)

Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

100%|████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 9.63MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1    990976  ultralytics.nn.modules.block.C2PSA           [512, 512, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1    443776  ultralytics.nn.modules.block.C3k2            [768, 256, 1, False]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    127680  ultralytics.nn.modules.block.C3k2            [512, 128, 1, False]          
 17                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128,

100%|██████████████████████████████████████████████████████████| 5.35M/5.35M [00:00<00:00, 21.5MB/s]


...

                   all        468        447      0.997      0.993      0.995        0.8



100 epochs completed in 0.713 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 19.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████


                   all        468        447      0.986      0.987      0.995      0.804
                 plane        350        350      0.991      0.984      0.995      0.784
           heliconpter         97         97      0.981       0.99      0.995      0.825
Speed: 0.2ms preprocess, 1.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4197500220>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [57]:
model = YOLO("yolo11s.yaml")
model.train(data="data.yaml", epochs=100, batch=-1, plots=True, save=True)

Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.yaml, data=data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

train: Scanning /home/user1/datasets/baseline_dataset/train/labels.cache... 3591 images, 172 backgro
val: Scanning /home/user1/datasets/baseline_dataset/valid/labels.cache... 447 images, 21 backgrounds


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.00059375), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100        19G      5.425      178.3      3.947         58        640: 100%|██████████| 50/5
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████

                   all        468        447          0          0          0          0


...

                   all        468        447      0.991      0.993      0.994      0.802



100 epochs completed in 0.721 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 19.2MB

Validating runs/detect/train4/weights/best.pt...
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████


                   all        468        447      0.991      0.993      0.994      0.801
                 plane        350        350      0.997      0.986      0.995      0.769
           heliconpter         97         97      0.984          1      0.994      0.833
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4197ca10f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
model = YOLO("yolo11s.pt")
model.tune(data="data.yaml", epochs=30, batch=-1, fraction=0.2, iterations=300, optimizer="AdamW", plots=False, save=False, val=False)

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune3'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/300 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=data.yaml, epochs=30, time=None, patience=100, batch=-1, imgsz=640, save=False, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=

train: Scanning /home/user1/datasets/baseline_dataset/train/labels.cache... 627 images, 126 backgrounds, 0 corrupt: 100%|██████████| 753/753 [00:00<?, ?it/s]
val: Scanning /home/user1/datasets/baseline_dataset/valid/labels.cache... 447 images, 21 backgrounds, 0 corrupt: 100%|██████████| 468/468 [00:00<?, ?it/s]


optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0006015625), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train7
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      19.3G        2.5      28.68      1.149         76        640: 100%|██████████| 10/10 [00:11<00:00,  1.15s/it]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      22/30      19.4G      1.404     0.5718     0.8305         53        640: 100%|██████████| 10/10 [00:04<00:00,  2.29it/s]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      23/30      19.4G       1.32     0.5392     0.8268         51        640: 100%|██████████| 10/10 [00:04<00:00,  2.28it/s]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      24/30      19.4G      1.301     0.5366      0.823         68        640:  90%|█████████ | 9/10 [00:04<00:00,  2.26it/s]

In [4]:
model = YOLO("yolo11s.pt")
model.train(data="data.yaml", epochs=100, batch=-1, plots=True, save=True,
           lr0= 0.00746,
            lrf= 0.00942,
            momentum= 0.8946,
            weight_decay= 0.00058,
            warmup_epochs= 2.92744,
            warmup_momentum= 0.71774,
            box= 8.4543,
            cls= 0.48317,
            dfl= 1.32973,
            hsv_h= 0.0161,
            hsv_s= 0.9,
            hsv_v= 0.60851,
            degrees= 0.0,
            translate= 0.12099,
            scale= 0.53349,
            shear= 0.0,
            perspective= 0.0,
            flipud= 0.0,
            fliplr= 0.41098,
            bgr= 0.0,
            mosaic= 0.91628,
            mixup= 0.0,
            copy_paste= 0.0,
)

Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train67, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /home/user1/datasets/baseline_dataset/train/labels... 3591 images, 172 backgrounds, 


train: New cache created: /home/user1/datasets/baseline_dataset/train/labels.cache


val: Scanning /home/user1/datasets/baseline_dataset/valid/labels.cache... 447 images, 21 backgrounds


Plotting labels to runs/detect/train67/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.00746' and 'momentum=0.8946' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0006978125), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train67
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        468        447      0.992      0.992      0.995      0.796



100 epochs completed in 0.779 hours.
Optimizer stripped from runs/detect/train67/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train67/weights/best.pt, 19.2MB

Validating runs/detect/train67/weights/best.pt...
Ultralytics 8.3.10 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla V100-SXM3-32GB, 32494MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████


                   all        468        447      0.991      0.992      0.994      0.801
                 plane        350        350      0.997      0.984      0.995      0.773
           heliconpter         97         97      0.985          1      0.994      0.829
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train67


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3617c320b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804